In [1]:
import urllib.request
import re
import pickle

# Crawler de Wikipedia

## Funciones auxiliares

La siguiente función toma como parámetro de entrada un url correspondiente a una página de Wikipedia, hace un request al servidor, y devuelve el código html de la misma como texto.

In [2]:
def gets_html_wikipedia(url):
    """ Lee un url y devuelve el código html """
    req = urllib.request.Request(url,
                                 headers={'User-Agent':
                                          'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})

    return(urllib.request.urlopen(req).read().decode("utf-8"))

La siguiente función toma como parámetros de entrada el código html de una página (como texto) y un entero que representa la cantidad de links a devolver, y devuelve una lista con los urls en formato string

In [3]:
def gets_n_wiki_links(html, n_links):
    """ Devuelve los primeros n links de un artículo """
    links = re.compile('<a href="/wiki\S*" title.*?>', re.U).findall(html)
    out = []

    while links and len(out) < n_links:
        li = links.pop(0)
        if li.count("=") == 2 and ":" not in li:
            link_tmp = "http://es.wikipedia.org" + re.compile('href="(.+?)"').findall(li)[0]
            if "#" in link_tmp:
                link_tmp = link_tmp.split("#")[0]
            out.append(link_tmp)

    return(out)

### Prueba de funciones auxiliares

In [4]:
to_scrap = [("http://es.wikipedia.org/wiki/Uruguay", 0),
            ("http://es.wikipedia.org/wiki/Pangea", 0)]
print(to_scrap)

[('http://es.wikipedia.org/wiki/Uruguay', 0), ('http://es.wikipedia.org/wiki/Pangea', 0)]


In [5]:
print(to_scrap[0][0])

http://es.wikipedia.org/wiki/Uruguay


In [6]:
print(gets_html_wikipedia(to_scrap[0][0]))

<!DOCTYPE html>
<html class="client-nojs" lang="es" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Uruguay - Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Uruguay","wgTitle":"Uruguay","wgCurRevisionId":120255939,"wgRevisionId":120255939,"wgArticleId":6216,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Páginas que utilizan Timeline","Wikipedia:Páginas con referencias web sin URL","Wikipedia:Artículos con enlaces externos rotos","Wikipedia:Páginas con referencias con parámetros obsoletos","Wikipedia:Artículos con datos por trasladar a Wikidata","Wikipedia:Artículos de países que requieren mantenimiento","Wikipedia:Artículos con datos locales","Wikipedia:Artículos destacados en la Wikipedia en catalán","Wikipedia:Artículos destacados en la Wikipedia en asturiano","Wi

In [7]:
print(gets_n_wiki_links(gets_html_wikipedia(to_scrap[0][0]), n_links=2))

['http://es.wikipedia.org/wiki/Bandera_de_Uruguay', 'http://es.wikipedia.org/wiki/Libertad_o_Muerte']


# Función principal: Scraper

Implemente una función que data una lista de links semilla (*to_scrap*) navegue wikipedia considerando los primeros 4 links de cada página (que no hayan sido ya visitados) y hasta una profundidad igual a 3.

La función deberá devolver un diccionario, en donde la clave del mismo sea el url de una página visitada y el valor sea el código html de dicha página.

In [8]:
def scraper(to_scrap, n_links, max_depth):

    html_data = {}
    
    while to_scrap:
        url, depth = to_scrap.pop()
        if (url not in html_data) and (depth <= max_depth):
            print((url, depth))
            html_data[url] = gets_html_wikipedia(url)
            page_urls = gets_n_wiki_links(html_data[url], n_links)
            page_urls = [(e, depth + 1) for e in page_urls]
            to_scrap.extend(page_urls)
            
    return(html_data)

## Casos de prueba

Las siguientes líneas de código servirán para ir evaluando el avance de su implementación. Si implementaron bien la función *scraper*, la úlitma línea debería dar como resultado el diccionario pedido.

### Prueba de función principal a implementar

In [9]:
html_data = scraper(to_scrap, n_links=4, max_depth=3)
print(html_data)

('http://es.wikipedia.org/wiki/Pangea', 0)
('http://es.wikipedia.org/wiki/Era_Mesozoica', 1)
('http://es.wikipedia.org/wiki/Era_Cenozoica', 2)
('http://es.wikipedia.org/wiki/E%C3%B3n_Fanerozoico', 3)
('http://es.wikipedia.org/wiki/Era_geol%C3%B3gica', 3)
('http://es.wikipedia.org/wiki/E%C3%B3n_(geolog%C3%ADa)', 3)
('http://es.wikipedia.org/wiki/Era_Paleozoica', 1)
('http://es.wikipedia.org/wiki/Supercontinente', 1)
('http://es.wikipedia.org/wiki/Crat%C3%B3n', 2)
('http://es.wikipedia.org/wiki/Ma_(unidad_de_tiempo)', 3)
('http://es.wikipedia.org/wiki/Provincia_geol%C3%B3gica', 3)
('http://es.wikipedia.org/wiki/Geolog%C3%ADa', 2)
('http://es.wikipedia.org/wiki/Continente', 2)
('http://es.wikipedia.org/wiki/Tierra', 3)
('http://es.wikipedia.org/wiki/Territorio', 3)
('http://es.wikipedia.org/wiki/Richard_Buckminster_Fuller', 3)
('http://es.wikipedia.org/wiki/Mapa_Dymaxion', 3)
('http://es.wikipedia.org/wiki/Uruguay', 0)
('http://es.wikipedia.org/wiki/Himno_nacional_de_Uruguay', 1)
('http:/

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Guardado de los datos en un archivo *pickle*

In [10]:
with open("html_data.p", "wb") as p:
    pickle.dump(html_data, p)